# Tabular Learning and the Bellman Equation
Given a MDP $(S, A, T, R, \gamma)$, the optimal state values $V(s)$ and state action values $Q(s, a)$ are described by the *Bellman Equation*:
$$
V(s) = \max_{a \in A} Q(s, a) = \max_{a \in A} \sum_{s' \in S} T(s, a, s') (R(s, a, s') + \gamma V(s'))
$$
where
* $T(s, a, s') \in [0, 1]$ is a stochastic transition model, i.e. the probability $P(s' | s, a)$ of reaching state $s' \in S$ from $s \in S$ with action $a \in A$
* $R(s, a, s')$ is the immediate reward of transitioning from $s$ to $s'$ using action $a$. Note that this may collapse, e.g. to $R(s')$.
* $\gamma \in (0, 1]$ is a discount factor for future rewards

Deterministic optimal policy then plays action $\pi(s) = argmax_{a \in A} Q(s, a)$. A deterministic policy can be found given complete knowledge of the environment, that is the transition model $T$ and rewards $R$. Otherwise one has to either build up a model of these or use other learning techiques.

*Value iteration* method then simply starts with $V_0(s) \gets 0$ and uses the Bellman update to compute $V_{k + 1}(s)$ using $V_k(s)$. *Q-value iteration* then simply adopts this approach for Q values (note that $V(s') = \max_{a'} Q(s', a')$ in the Bellman update).

## FrozenLake: Value Iteration

In [1]:
from dataclasses import dataclass

import gym
import numpy as np
from tensorboardX import SummaryWriter


@dataclass
class EnvModel:
    transits: np.ndarray
    rewards: np.ndarray

    @classmethod
    def new(cls, n_states: int, n_actions: int) -> "EnvModel":
        shape = (n_states, n_actions, n_states)
        return cls(
            transits=np.zeros(shape=shape, dtype=np.uint32),
            rewards=np.zeros(shape=shape, dtype=np.float64),
        )

    def add_experience(self, s: int, a: int, r: float, sp: int) -> None:
        self.transits[s, a, sp] += 1
        self.rewards[s, a, sp] = r

    @property
    def tranistion_probas(self) -> np.ndarray:
        """
        Computes empirical distribution over next states for each (s, a).

        Note: Transitions (s, a) that have not yet been played are set to 0.
        """
        totals = np.sum(self.transits, axis=2)[..., np.newaxis]
        probas = np.zeros_like(self.transits, dtype=np.float64)
        return np.divide(self.transits, totals, out=probas, where=totals > 0)


class Agent:
    def __init__(
        self,
        state: int,
        n_states: int,
        n_actions: int,
        gamma: float,
    ) -> None:
        self.state = state
        self.model = EnvModel.new(n_states, n_actions)
        self.gamma = gamma
        self.values = np.zeros(n_states, dtype=np.float64)

    @classmethod
    def init(cls, env: gym.Env, gamma: float) -> "Agent":
        return cls(
            state=env.reset(),
            n_states=env.observation_space.n,
            n_actions=env.action_space.n,
            gamma=gamma,
        )

    @property
    def q_values(self) -> np.ndarray:
        targets = self.model.rewards + self.gamma * self.values
        return np.sum(self.model.tranistion_probas * targets, axis=2)

    def policy(self, state: int) -> int:
        return np.argmax(self.q_values[state])

    def explore(self, env: gym.Env, n_steps: int) -> None:
        """
        Explore given environment for a bit and update policy
        based on collected experience.
        """

        # Improve the environment model based on experiences
        for _ in range(n_steps):

            # Use random policy for exploration
            action = env.action_space.sample()

            # Gain new experience by interacting with the environment
            next_state, reward, done, _ = env.step(action)
            self.model.add_experience(self.state, action, reward, next_state)

            # Update internal state
            self.state = env.reset() if done else next_state

        # Adjust values using the Bellman update
        self.values = np.max(self.q_values, axis=1)

    def play_episode(self, env: gym.Env) -> float:
        state = env.reset()
        total_reward = 0.0
        done = False

        while not done:
            # Use learned policy to select best action in current state
            action = self.policy(state)

            # Apply the action and collect new experience
            next_state, reward, done, _ = env.step(action)
            self.model.add_experience(state, action, reward, next_state)

            # Accumulate non-discounted reward for the episode
            total_reward += reward
            state = next_state

        return total_reward

    def exploit(self, env: gym.Env, n_episodes: int) -> float:
        """
        Play several episodes in given environment and return
        average non-discounted reward.

        Note: This does not update agent's state, except the environment model
        """
        total_reward = sum(self.play_episode(env) for _ in range(n_episodes))
        return total_reward / n_episodes


# Hyperparameters
TEST_EPISODES = 20
EXPLORE_STEPS = 100
SOLUTION_BOUND = 0.8
MAX_ITERS = 1_000
GAMMA = 0.9

# Crate two environments
#  - First one is for exploration and training
#  - The other is a sandpox for evaluation
env = gym.make("FrozenLake8x8-v0")
test_env = gym.make("FrozenLake8x8-v0")

# Initialize an agent
agent = Agent.init(env, gamma=GAMMA)

with SummaryWriter(comment="-v-iteration") as writer:

    i = 0
    reward = 0.0
    best_reward = 0.0

    while reward < SOLUTION_BOUND and i < MAX_ITERS:
        i += 1

        # Improve the policy by interacting with the environment
        agent.explore(env=env, n_steps=EXPLORE_STEPS)

        # Run some trials in sandboxed environment
        #  - Note: We collect new experience here too.
        reward = agent.exploit(env=test_env, n_episodes=TEST_EPISODES)

        # Update best overall reward
        best_reward = max(best_reward, reward)

        # Record rewards for monitoring
        writer.add_scalar("reward", reward, i)
        writer.add_scalar("best_reward", best_reward, i)

# Show final result
print(f"Solved in {i} iterations with best reward: {best_reward:.3f}")

Solved in 131 iterations with best reward: 0.800
